# FAHASA

## Khởi Tạo

In [40]:
%pip install beautifulsoup4 numpy pandas requests selenium matplotlib seaborn regex geckodriver-autoinstaller webdriver-manager

In [41]:
from urllib.request import urlopen
import urllib.request as urllib2
from urllib.error import HTTPError
from bs4 import BeautifulSoup, SoupStrainer
import numpy as np
import pandas as pd
import concurrent.futures
from multiprocessing import Pool
import requests
from selenium import webdriver
from selenium.common.exceptions import *
import matplotlib.pyplot as plt
import seaborn as sns
from selenium.webdriver.common.by import By
import argparse
from datetime import datetime
import os
import re
import time
import argparse
from collections import Counter
from datetime import datetime
import os
import regex as re
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException, ElementNotVisibleException, StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from urllib.request import urlopen
from urllib.error import HTTPError
from selenium.webdriver.common.by import By
import pandas as pd
import geckodriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager

## MAIN

In [42]:
pages = ['https://www.fahasa.com/sach-trong-nuoc/van-hoc-trong-nuoc/tieu-thuyet.html'] + \
    [f'https://www.fahasa.com/sach-trong-nuoc/van-hoc-trong-nuoc/tieu-thuyet.html?order=num_orders&limit=24&p={i}' for i in range(2, 87)]


In [43]:
import concurrent.futures

bookTitles = []
images = []
urls = []
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}

def take_bookTitles_and_images(page):
    r = requests.get(page, headers=headers)
    r.raise_for_status()
    soup = BeautifulSoup(r.content, 'html.parser')

    titles = [img_tag.find('img')['alt'] for img_tag in soup.find_all('span', class_='product-image')]
    imgs = [img_tag.find('img')['data-src'] for img_tag in soup.find_all('span', class_='product-image')]
    page_urls = [a_tag['href'] for a_tag in soup.find_all('a', class_='product-image')]

    return titles, imgs, page_urls

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(take_bookTitles_and_images, pages))

for titles, imgs, page_urls in results:
    bookTitles.extend(titles)
    images.extend(imgs)
    urls.extend(page_urls)

In [44]:
print(len(urls))

1860


In [45]:
wantedBooks = len(urls)

In [46]:
prices = []
details = []
genres = []

In [ ]:
def extract_text_or_default(soup, selector, default="No detail"):
    element = soup.select_one(selector)
    return element.text if element else default

def extract_genres(soup):
    genresRaw = soup.select('ol.breadcrumb a')
    return [genre.text for genre in genresRaw] if genresRaw else ["No genre"]

def take_prices_inf_genre(session, url):
    r = session.get(url, headers=headers)
    r.raise_for_status()
    soup = BeautifulSoup(r.content, 'html.parser')

    price = extract_text_or_default(soup, 'span.price', "No price")
    detail = extract_text_or_default(soup, 'div#desc_content', "No detail")
    genre = extract_genres(soup)

    return price, detail, genre

def process_url(session, url):
    try:
        return take_prices_inf_genre(session, url)
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return "No price", "No detail", ["No genre"]

with requests.Session() as session:
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(lambda url: process_url(session, url), urls))

for price, detail, genre in results:
    prices.append(price)
    details.append(detail)
    genres.append(genre)


In [ ]:
print(len(bookTitles))
print(len(prices))
print(len(details))
print(len(images))
print(len(genres))

In [ ]:
pd.DataFrame({"Ten ": bookTitles,"Gia ": prices ,"Thong tin": details,"Anh":images,"The loai":genres}).to_csv("data_fahasa.csv")

## User

In [ ]:
# take the data from the csv file
data = pd.read_csv("data_fahasa.csv")
data = data.drop(columns=["Unnamed: 0"])
data = data.dropna()
data = data.reset_index(drop=True)
data


In [ ]:
bookTitles = data["Ten "]
prices = data["Gia "]
details = data["Thong tin"]
images = data["Anh"]
genres = data["The loai"]

In [ ]:
import re

In [ ]:
bookTitlesWithoutToneMark = []

In [ ]:
import concurrent.futures

def no_accent_vietnamese(s):
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'I', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    return s

def separate(title):
    return title.split('(')[0].split('-')[0]

goodReadsOriginalLink = 'https://www.goodreads.com/'
goodReadsOriginalSearchLink = 'https://www.goodreads.com/search?q='
goodReadsLinkList = []

bookTitlesWithoutToneMark = [separate(bookTitle) for bookTitle in bookTitles]

def fetch_goodreads_link(bookTitleWithoutToneMark):
    urlForBookTitleInGoodReads = goodReadsOriginalSearchLink + '%20'.join(bookTitleWithoutToneMark.split())
    urlForBookTitleInGoodReads = no_accent_vietnamese(urlForBookTitleInGoodReads)
    try:
        html = urlopen(urlForBookTitleInGoodReads)
        bs = BeautifulSoup(html.read(), 'html.parser')
        bookTitle = bs.find('a', class_='bookTitle')
        return goodReadsOriginalLink + bookTitle['href']
    except Exception as e:
        return ''

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    goodReadsLinkList = list(executor.map(fetch_goodreads_link, bookTitlesWithoutToneMark))


In [ ]:
count = 0
for link in goodReadsLinkList:
  if (link == ''):
    count = count + 1

print(count)

In [ ]:
print(len(goodReadsLinkList))
print(goodReadsLinkList)

In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import concurrent.futures

inf = []
rating = []
all_genres = []

def extract_data(soup, selector, attr=None, default=""):
    element = soup.select_one(selector)
    if element:
        return element[attr] if attr else element.text.strip()
    return default

def genre_about(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        rate_dict = {'5': 0, '4': 0, '3': 0, '2': 0, '1': 0}
        json_data = soup.find('script', type='application/ld+json')
        if json_data:
            data = json.loads(json_data.string)
            if "aggregateRating" in data:
                rate_dict = data['aggregateRating']['ratingValue']

        desc_text = extract_data(soup, 'div.DetailsLayoutRightParagraph__widthConstrained')

        genres = [genre_link.text.strip() for genre_link in soup.select('div[data-testid="genresList"] a')]
        genres = list(set(genres)) if genres else []

        return desc_text, rate_dict, genres

    except Exception as e:
        return '', '', []

# Sử dụng ThreadPoolExecutor để thực hiện các yêu cầu song song
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(genre_about, goodReadsLinkList))

# Tách kết quả ra thành các danh sách riêng biệt
for desc_text, rate_dict, genres_list in results:
    inf.append(desc_text)
    rating.append(rate_dict)
    all_genres.append(genres_list)

# Thực thi và in kết quả
print(inf)
print(rating)
print(all_genres)


In [ ]:
dict_id_rating_ratingBook = [{}]
options = webdriver.FirefoxOptions()
options.headless = True

def main(url, book):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36'
    }

    retries = 3
    for _ in range(retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            scriptData = soup.find('script', id="__NEXT_DATA__")
            for i in json.loads(scriptData.string)['props']['pageProps']['apolloState']:
                if (json.loads(scriptData.string)['props']['pageProps']['apolloState'][i]['__typename'] == 'Review'):
                    temp = json.loads(scriptData.string)['props']['pageProps']['apolloState'][i]
                    dict_id_rating_ratingBook.append({'id': temp['creator']['__ref'].replace('User:', ''), 'bookName': book, 'rating': temp['rating'], 'ratingBook': temp['text']})
            break
        except requests.exceptions.HTTPError as e:
            time.sleep(2)
        except Exception as e:
            break

def process_books(urls, books):
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        executor.map(lambda p: main(*p), zip(urls, books))

process_books(goodReadsLinkList, bookTitles)

In [ ]:
user_id = []
user_rating = []
user_rating_book = []
user_review = []
for i in range(1,len(dict_id_rating_ratingBook)):
    user_id.append(dict_id_rating_ratingBook[i]['id'])
    user_rating.append(dict_id_rating_ratingBook[i]['rating'])
    user_rating_book.append(dict_id_rating_ratingBook[i]['bookName'])
    user_review.append(dict_id_rating_ratingBook[i]['ratingBook'])
print(len(user_id))
print(len(user_rating))
print(len(user_rating_book))
print(len(user_review))

In [ ]:
from collections import defaultdict

book_user_dict = defaultdict(list)
for entry in dict_id_rating_ratingBook:
    if 'bookName' in entry and 'id' in entry:
        book_user_dict[entry['bookName']].append(entry['id'])
rating_users_for_each_books = [book_user_dict[book] for book in bookTitles]

In [ ]:
len(rating_users_for_each_books)

In [ ]:
pd.DataFrame({"id ": user_id,"rating ": user_rating ,"name": user_rating_book,"Review": user_review}).to_csv("review_data.csv")
pd.DataFrame({'name':bookTitles,'thong tin':inf,'rating':rating,'genre':all_genres,'rating_users':rating_users_for_each_books}).to_csv("good_read.csv")

# Gộp

In [ ]:
fhs = pd.read_csv('data_fahasa.csv')
gr = pd.read_csv('good_read.csv')
rd = pd.read_csv('review_data.csv')

In [ ]:
print(fhs)

In [ ]:
print(gr)

In [ ]:
print(rd)

In [ ]:
def no_accent_vietnamese(s):
    s = re.sub('\xa0', ' ', s)
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'I', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    return s

def twodtooned(m):
    return [no_accent_vietnamese(lt) if isinstance(lt, str) else '' for lt in m]

list_name = twodtooned(fhs['Ten '])
temp = twodtooned(rd['name'])


In [ ]:
list_inf = []
list_genres = []
real_name = []
rdid = []
rdrating = []
list_img = []

def find_index(theList, item):
    for ind, val in enumerate(theList):
        if item in val:
            return ind
    return 'Not Found'

def append_data(res, i):
    if res != 'Not Found':
        list_img.append(fhs['Anh'][res])
        list_inf.append(fhs['Thong tin'][res])
        list_genres.append(gr['genre'][res])
        real_name.append(fhs['Ten '][res])
    else:
        list_inf.append('')
        list_genres.append('')
        list_img.append('')
        real_name.append('')

for i, item in enumerate(temp):
    res = find_index(list_name, item)
    rdid.append(rd['id '][i])
    rdrating.append(rd['rating '][i])
    append_data(res, i)

st = gr['rating_users']
for i in range(len(st)):
    if isinstance(st[i], float):
        list_img.append(fhs['Anh'][i])
        real_name.append(gr['name'][i])
        list_inf.append(gr['Thong tin'][i])
        list_genres.append(gr['genre'][i])
        rdid.append('')
        rdrating.append('')

In [ ]:
pd.DataFrame({'id':rdid,'rating':rdrating,'name':real_name,'Hinh':list_img,'Thong tin':list_inf,'The Loai':list_genres}).to_csv('All.csv')

In [ ]:
fileCSV = pd.read_csv("All.csv")
pd.DataFrame({'User-ID':fileCSV['id']}).to_csv('Users.csv')
pd.DataFrame({'User-ID':fileCSV['id'],'Book-Rating':fileCSV['rating']}).to_csv('Ratings.csv')
pd.DataFrame({'Book-Title':fileCSV['name'],'Image':fileCSV['Hinh'],'Description':fileCSV['Thong tin'],'Genres':fileCSV['The Loai']}).to_csv('Books.csv')

In [ ]:
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

In [ ]:
books

In [ ]:
ratings

In [ ]:
users